In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [2]:
# Ambulatory MDRO Summary Files
MDRO_Summary = pd.read_csv("Amb MDRO Summary.csv")
# HAI Individual Patient Files
MDRO_Individual = pd.read_csv("Amb MDRO Individuals.csv")

In [3]:
# convert the 'SpecimenDate' column to datetime format
MDRO_Individual.SpecimenDate = pd.to_datetime(MDRO_Individual.SpecimenDate)
# Change the date formating
MDRO_Individual.SpecimenDate = MDRO_Individual.SpecimenDate.apply(lambda x: x.strftime('%m/%d/%Y'))

## Ambulatory HAI Summary Function

In [4]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Amb_HAI_Summary_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to add or create each HAI summary section
    
    # This Hospital's Summary Data
    Hosp_MDRO_data = MDRO_Summary[MDRO_Summary.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_MDRO_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct year and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # Take the xml UnitName and replace it with the correct Hospital UnitName from the Hosp_Units list. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]
        current_xml_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # With this correct EUOSH UnitName grab each of the 3 months for that unit from the EUH data
        MDRO_new_data = Hosp_MDRO_data[Hosp_MDRO_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        MDRO_new_data = MDRO_new_data.sort_values('Month')


        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = MDRO_new_data.Month.iloc[idx2]

            # Take each xml MDROSummary
            MDRO_nodes = month.getElementsByTagName("MDROAmbulatorySummary")
            # Now replace each xml metric value with data that has the same name from our data source
            for MDRO_node in MDRO_nodes:

                for column in MDRO_new_data.columns[3:]:
                    new_value = MDRO_new_data[column].iloc[idx2]
                    MDRO_node.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



     # This is a temporary xml file that I will now use to add the individual separation data.
    tree.writexml(open('Temp_{}.xml'.format(Selected_Hosp), 'w'))

In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital Temp XML file,
# Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Amb_HAI_Indiv_XML(Selected_Hosp,Temp_XML,Calendar_Year,Calendar_Quarter):
    # This is the temporary xml file that has already had the HAI Summary section filled
    
    # This Hospital's Individual Data
    Hosp_MDRO_individual = MDRO_Individual[MDRO_Individual.Hospital == Selected_Hosp]
    
    tree = xml.dom.minidom.parse(Temp_XML)
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # This loops through each listed unit in our Temp xml file
    for idx, unit in enumerate(unit_nodes):
        each_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Then we loop through each listed month
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            each_month = month.getElementsByTagName('Month')[0].childNodes[0].nodeValue

            CDIFF_count = month.getElementsByTagName('TotalCDiff')[0].childNodes[0].nodeValue
            MDRA_count = month.getElementsByTagName('TotalMRSA')[0].childNodes[0].nodeValue     

            MDROs = [int(CDIFF_count), int(MDRA_count)]
            total_MDROs = sum(MDROs)

            if total_MDROs > 0:
                # Then I grab each patient with HAI data for this unit
                MDRO_patients = Hosp_MDRO_individual[Hosp_MDRO_individual.UnitName == each_unit]
                # Lastly, I select only that unit's HAIs that occured in each specific month
                Monthly_MDROs = MDRO_patients[MDRO_patients.Month == int(each_month)]

                # Now I can take each listed HAI
                for i in range(len(Monthly_MDROs)):

                    # and create a "MDROEvent" node for each individual MDRO
                    MDRO_patient = tree.createElement("MDROAmbulatoryEvent")

                    MDRO_patient_list = ['AgeYears','xGenderFID','xMDROOrganismFID','SpecimenDate']
                    for data in MDRO_patient_list:
                        # I create an element for each individual's metrics
                        data_name = tree.createElement(data)
                        data_name.appendChild(tree.createTextNode(str(Monthly_MDROs[data].iloc[i])))

                        MDRO_patient.appendChild(data_name)

                    # Finally, I can add these individual MDRO patients to the "MDROEvents" section.
                    month.getElementsByTagName("MDROAmbulatoryEvents")[0].appendChild(MDRO_patient)


    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Amb MDRO Data.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [6]:
NDNQI_Amb_HAI_Summary_XML('EUOSH','2020 EUOSH Amb MDRO Template.xml','SBI556',2021,2)

In [7]:
NDNQI_Amb_HAI_Indiv_XML('EUOSH','Temp_EUOSH.xml',2021,2)

## EJCH

In [8]:
NDNQI_Amb_HAI_Summary_XML('EJCH','2020 EJCH Amb MDRO Template.xml','YTD069',2021,2)

In [9]:
NDNQI_Amb_HAI_Indiv_XML('EJCH','Temp_EJCH.xml',2021,2)

## ESJH

In [10]:
NDNQI_Amb_HAI_Summary_XML('ESJH','2020 ESJH Amb MDRO Template.xml','XAR317',2021,2)

In [11]:
NDNQI_Amb_HAI_Indiv_XML('ESJH','Temp_ESJH.xml',2021,2)

## EUHM

In [12]:
NDNQI_Amb_HAI_Summary_XML('EUHM','2020 EUHM Amb MDRO Template.xml','PVF155',2021,2)

In [13]:
NDNQI_Amb_HAI_Indiv_XML('EUHM','Temp_EUHM.xml',2021,2)

## EUH

In [14]:
NDNQI_Amb_HAI_Summary_XML('EUH','2020 EUH Amb MDRO Template.xml','JXE432',2021,2)

In [15]:
NDNQI_Amb_HAI_Indiv_XML('EUH','Temp_EUH.xml',2021,2)

## EDH

In [16]:
NDNQI_Amb_HAI_Summary_XML('EDH','2021 EDH Amb MDRO Template.xml','NMW346',2021,2)

In [17]:
NDNQI_Amb_HAI_Indiv_XML('EDH','Temp_EDH.xml',2021,2)

## EHH

In [18]:
NDNQI_Amb_HAI_Summary_XML('EHH','2021 EHH Amb MDRO Template.xml','WXX116',2021,2)

In [19]:
NDNQI_Amb_HAI_Indiv_XML('EHH','Temp_EHH.xml',2021,2)